In [1]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, modified_precision
from nltk.translate.chrf_score import sentence_chrf, corpus_chrf
from nltk.metrics import scores
import scipy.io.wavfile
from IPython.display import Audio
from IPython.display import display
from nltk.stem import *
# from nltk.stem.snowball import SnowballStemmer
from stemming.porter2 import stem
import stemming
from nltk.metrics.scores import recall

from basics import *

from nltk.corpus import stopwords

%matplotlib inline

In [2]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)

In [3]:
smooth_fun = nltk.translate.bleu_score.SmoothingFunction()

In [4]:
from nmt_run import *

In [5]:
cfg_path = "interspeech_new_vocab/sp_160hrs/"

In [6]:
%%capture
last_epoch, model, optimizer, m_cfg, t_cfg = check_model(cfg_path)

### Load Fisher dataset

In [7]:
%%capture
# -------------------------------------------------------------------------
# get data dictionaries
# -------------------------------------------------------------------------
map_dict, vocab_dict, bucket_dict = get_data_dicts(m_cfg)
info_dict = pickle.load(open("fbanks_80dim_nltk/info.dict", "rb"))
sim_dict = pickle.load(open("./fbanks_80dim_nltk/mix_sim.dict", "rb"))

In [8]:
random.seed("meh")
# random.seed("haha")

### Train text

In [ ]:
train_file = "../installs/fisher-callhome-corpus/corpus/ldc/fisher_train.en"

In [ ]:
train_text = []

In [ ]:
for u in map_dict["fisher_train"]:
    train_text.append(" ".join([w.decode() for w in map_dict["fisher_train"][u]["en_w"]]))

In [ ]:
train_text[:10]

In [ ]:
train_text_to_dump = "\n".join(train_text)

In [ ]:
with open("../subword-nmt/fisher_train.en", "w") as out_f:
    out_f.write(train_text_to_dump)

In [ ]:
# ./learn_joint_bpe_and_vocab.py --input {train_file}.L1 {train_file}.L2 -s {num_operations} -o {codes_file} --write-vocabulary {vocab_file}.L1 {vocab_file}.L2

In [ ]:
# sp = spm.SentencePieceProcessor()

In [ ]:
# sp.Load("test/test_model.model")

### word level analysis

In [9]:
min_word_len = 1
top_k = 100

In [10]:
stop_words = set(nltk.corpus.stopwords.words("english"))
len(stop_words)

127

In [11]:
es_stop_words = set(nltk.corpus.stopwords.words("spanish"))
len(es_stop_words)

313

In [12]:
def get_words(m_dict, key="en_w"):
    words = []
    for u in m_dict:
        if type(m_dict[u][key]) == list:
            words.extend([w.decode() for w in m_dict[u][key]])
        else:
            for ref in m_dict[u][key]:
                words.extend([w.decode() for w in ref])
    return Counter(words)

In [13]:
len(map_dict['fisher_dev'])

3979

In [14]:
# words in train
train_words = get_words(map_dict['fisher_train'])
train_words_top_k = [(w,f) for w, f in sorted(train_words.items(), reverse=True, key=lambda t:t[1]) 
                     if w not in stop_words and len(w) >= min_word_len][:top_k]

train_only_words = set(train_words.keys())

print("{0:20s} | {1:10d}".format("# train word types", len(train_words)))
print("{0:20s} | {1:10d}".format("# train word tokens", sum(train_words.values())))

dev_words = get_words(map_dict['fisher_dev'])
dev_words_top_k = [(w,f) for w, f in sorted(dev_words.items(), reverse=True, key=lambda t:t[1]) 
                     if w not in stop_words and len(w) >= min_word_len][:top_k]

dev_only_words = set(dev_words.keys())

print("-"*80)
print("{0:20s} | {1:10d}".format("# dev word types", len(dev_words)))
print("{0:20s} | {1:10d}".format("# dev word tokens", sum(dev_words.values())))

# train word types   |      17688
# train word tokens  |    1496533
--------------------------------------------------------------------------------
# dev word types     |       4798
# dev word tokens    |     165047


In [15]:
# words in train
es_train_words = get_words(map_dict['fisher_train'], key="es_w")
es_train_words_top_k = [(w,f) for w, f in sorted(es_train_words.items(), reverse=True, key=lambda t:t[1]) 
                     if w not in es_stop_words and len(w) >= min_word_len][:top_k]

es_train_only_words = set(es_train_words.keys())

print("{0:20s} | {1:10d}".format("# train word types", len(es_train_words)))
print("{0:20s} | {1:10d}".format("# train word tokens", sum(es_train_words.values())))

es_dev_words = get_words(map_dict['fisher_dev'], key="es_w")
es_dev_words_top_k = [(w,f) for w, f in sorted(es_dev_words.items(), reverse=True, key=lambda t:t[1]) 
                     if w not in es_stop_words and len(w) >= min_word_len][:top_k]

es_dev_only_words = set(es_dev_words.keys())

print("-"*80)
print("{0:20s} | {1:10d}".format("# dev word types", len(es_dev_words)))
print("{0:20s} | {1:10d}".format("# dev word tokens", sum(es_dev_words.values())))

# train word types   |      32168
# train word tokens  |    1494878
--------------------------------------------------------------------------------
# dev word types     |       4073
# dev word tokens    |      41014


In [16]:
train_words_top_k[:5], es_train_words_top_k[:5]

([('yes', 35085),
  ("'s", 24162),
  ("n't", 19184),
  ('like', 14337),
  ('well', 12354)],
 [('ah', 12598), ('eh', 11578), ('si', 9541), ('ajá', 7992), ('bueno', 7874)])

In [17]:
[(w,f) for w,f in train_words_top_k if "'" in w]

[("'s", 24162), ("n't", 19184), ("'m", 5546), ("'re", 2832), ("'ve", 2392)]

In [18]:
dev_words_top_k[:5], es_dev_words_top_k[:5]

([('yes', 3652), ("n't", 1999), ("'s", 1866), ('like', 1827), ('know', 1294)],
 [('ah', 349), ('ajá', 343), ('si', 251), ('entonces', 249), ('mhm', 236)])

In [19]:
oov_words = {w:f for w,f in dev_words.items() if w not in train_only_words}

print("{0:20s} | {1:10d}".format("# oov word types", len(oov_words)))
print("{0:20s} | {1:10d}".format("# oov word tokens", sum(oov_words.values())))

# oov word types     |        999
# oov word tokens    |       1562


In [20]:
es_oov_words = {w:f for w,f in es_dev_words.items() if w not in es_train_only_words}

print("{0:20s} | {1:10d}".format("# oov word types", len(es_oov_words)))
print("{0:20s} | {1:10d}".format("# oov word tokens", sum(es_oov_words.values())))

# oov word types     |        440
# oov word tokens    |        517


In [21]:
"{0:.1f}%".format(sum(oov_words.values()) / sum(dev_words.values()) * 100)

'0.9%'

### Word level - get train, dev frequency, and utts in which they occur

In [22]:
len(train_only_words), len(set([stem(w) for w in train_only_words]))

(17688, 11869)

In [23]:
def get_word_level_details(word_key):
    word_utt_count = {"train": {}, "dev": {}, "train_utts": {}, "dev_utts": {}}
    for u in tqdm(map_dict["fisher_train"].keys()):
        for w in set(map_dict["fisher_train"][u][word_key]):
            curr_word = w.decode()
            if curr_word not in word_utt_count["train"]:
                word_utt_count["train"][curr_word] = 0
                word_utt_count["train_utts"][curr_word] = set()
            word_utt_count["train"][curr_word] += 1
            word_utt_count["train_utts"][curr_word].update({u})
        # end for words in current utt
    # end for all utts
    for u in tqdm(map_dict["fisher_dev"].keys()):
        if word_key == "en_w":
            for ref in map_dict["fisher_dev"][u][word_key]:
                for w in set(ref):
                    curr_word = w.decode()
                    if curr_word not in word_utt_count["dev"]:
                        word_utt_count["dev"][curr_word] = 0
                        word_utt_count["dev_utts"][curr_word] = set()
                    word_utt_count["dev"][curr_word] += 1            
                    word_utt_count["dev_utts"][curr_word].update({u})
                # end for words in current ref
            # end for all references
        # end if multiple references
        else:
            ref = map_dict["fisher_dev"][u][word_key]
            for w in set(ref):
                curr_word = w.decode()
                if curr_word not in word_utt_count["dev"]:
                    word_utt_count["dev"][curr_word] = 0
                    word_utt_count["dev_utts"][curr_word] = set()
                word_utt_count["dev"][curr_word] += 1            
                word_utt_count["dev_utts"][curr_word].update({u})
            
    # end for all utts
    all_train_utts = set()
    for w in word_utt_count["train_utts"]:
        all_train_utts.update(word_utt_count["train_utts"][w])
    # end for

    all_dev_utts = set()
    for w in word_utt_count["dev_utts"]:
        all_dev_utts.update(word_utt_count["dev_utts"][w])
    # end for
    
    return word_utt_count, all_train_utts, all_dev_utts
    

In [24]:
en_word_utt_count, en_train_utts, en_dev_utts = get_word_level_details("en_w")

100%|██████████| 3979/3979 [00:00<00:00, 26484.60it/s]


In [25]:
len(en_train_utts), len(en_dev_utts)

(138794, 3979)

In [ ]:
es_word_utt_count, es_train_utts, es_dev_utts = get_word_level_details("es_w")

In [ ]:
len(es_train_utts), len(es_dev_utts)

In [19]:
print("word types")
print(len(en_word_utt_count['train']), len(en_word_utt_count['dev']))
print("common word types")
en_common_words = set(en_word_utt_count['train'].keys()) & set(en_word_utt_count['dev'].keys())
len(en_common_words)

word types
17688 4798
common word types


3799

In [20]:
print("word types")
print(len(es_word_utt_count['train']), len(es_word_utt_count['dev']))
print("common word types")
es_common_words = set(es_word_utt_count['train'].keys()) & set(es_word_utt_count['dev'].keys())
len(es_common_words)

word types


NameError: name 'es_word_utt_count' is not defined

In [26]:
def get_details_for_words(words, common_words, word_utt_count, 
                          min_dev_freq, max_dev_freq, min_train_freq, max_train_freq, min_len):
    details = {"words": {}, "train_utts": set(), "dev_utts": set()}
    
    in_vocab_words = set(words) & set(common_words)
    print("number of in-vocab words = {0:d}".format(len(in_vocab_words)))

    for w in in_vocab_words:
        t_count, d_count = len(word_utt_count["train_utts"][w]), len(word_utt_count["dev_utts"][w])
        if ((d_count >= min_dev_freq) and 
            (d_count <= max_dev_freq) and
            (len(w) >= min_len) and
            (t_count >= min_train_freq) and 
            (t_count <= max_train_freq)):
            details["words"][w] = {"train": t_count, "dev": d_count}
            details["train_utts"].update(word_utt_count["train_utts"][w])
            details["dev_utts"].update(word_utt_count["dev_utts"][w])
        # end meets criteria
    # end for in-vocab word
    return details
# end function

In [27]:
def get_duration(utts, key):
    dur = 0
    utts_not_found = []
    for u in utts:
        if u not in info_dict[key]:
            #print("argh!", u)
            utts_not_found.append(u)
        else:
            dur += (info_dict[key][u]['sp'] * 10)
    dur = dur / 60 / 60 / 1000
    print("-"*80)
    print("{0:d} total utts".format(len(utts)))
    print("{0:d} not found".format(len(utts_not_found)))
    print("selected utts from {0:s} -- duration = {1:.2f} hours".format(key, dur))
    return dur

In [23]:
def create_vocab(words_list):
    out = {"w2i":{}, "i2w":{}, "freq":{}, "freq_dev":{}}
    START_VOCAB = [PAD, GO, EOS, UNK]
    for w in START_VOCAB:
        out['w2i'][w] = len(out["w2i"])
        out["freq"][w] = 1
    #for w in words_list['words']:
    sorted_w = sorted(words_list['words'].items(), reverse=True, key=lambda t: t[1]['train'])
    for w in sorted_w:
        encoded_word = w[0].encode()
        out["w2i"][encoded_word] = len(out["w2i"])
        out["freq"][encoded_word] = w[1]["train"]
        out["freq_dev"][encoded_word] = w[1]["dev"]

    out["i2w"] = {val:key for key, val in out["w2i"].items()}
    return out

In [24]:
train_dur, dev_dur = get_duration(en_train_utts, key="fisher_train"), get_duration(en_dev_utts, key="fisher_dev")

--------------------------------------------------------------------------------
138794 total utts
89 not found
selected utts from fisher_train -- duration = 161.62 hours
--------------------------------------------------------------------------------
3979 total utts
2 not found
selected utts from fisher_dev -- duration = 4.35 hours


### Task 0 - 500 randomly selected frequent words, minor filtering

In [ ]:
min_dev_freq=10
max_dev_freq=10000
min_train_freq=50 
min_len=5

In [ ]:
terms_of_interest = get_details_for_words(common_words, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq, 
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(terms_of_interest["words"])))

In [ ]:
_, _ = get_duration(terms_of_interest["train_utts"], key="fisher_train"), get_duration(terms_of_interest["dev_utts"], key="fisher_dev")

In [ ]:
random.seed("hmm")
sample_terms = random.sample(list(terms_of_interest["words"].keys()), 500)

In [ ]:
sample_terms_details = get_details_for_words(sample_terms, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq, 
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(sample_terms_details["words"])))

In [ ]:
_, _ = get_duration(sample_terms_details["train_utts"], key="fisher_train"), get_duration(sample_terms_details["dev_utts"], key="fisher_dev")

In [ ]:
sample_terms[:10]

In [ ]:
bow_top_500_words_vocab = create_vocab(sample_terms_details)

In [ ]:
pickle.dump(bow_top_500_words_vocab, open(os.path.join(m_cfg['data_path'], "bow_top_500_words_vocab.dict"), "wb"))

### Task 1 - randomly selected frequent words

In [ ]:
min_dev_freq=10 
max_dev_freq=100
min_train_freq=100
min_len=5

In [ ]:
terms_of_interest = get_details_for_words(common_words, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq, 
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(terms_of_interest["words"])))

In [ ]:
_, _ = get_duration(terms_of_interest["train_utts"], key="fisher_train"), get_duration(terms_of_interest["dev_utts"], key="fisher_dev")

In [ ]:
random.seed("hmm")
sample_terms = random.sample(list(terms_of_interest["words"].keys()), 100)

In [ ]:
sample_terms_details = get_details_for_words(sample_terms, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq, 
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(sample_terms_details["words"])))

In [ ]:
_, _ = get_duration(sample_terms_details["train_utts"], key="fisher_train"), get_duration(sample_terms_details["dev_utts"], key="fisher_dev")

In [ ]:
sample_terms[:10]

In [ ]:
bow_top_100_words_vocab = create_vocab(sample_terms_details)

In [ ]:
pickle.dump(bow_top_100_words_vocab, open(os.path.join(m_cfg['data_path'], "bow_top_100_words_vocab.dict"), "wb"))

### Task 2 - topics as keywords

In [ ]:
train_text_fname= "../installs/fisher-callhome-corpus/corpus/ldc/fisher_train.en"
topics_fname = "../criseslex/fsp06_topics_in_english.txt"

In [ ]:
topics = [ "peace", "Music", "Marriage", "Religion", "Cell phones", 
           "Dating", "Telemarketing and SPAM", "Politics", "Travel", 
           "Technical devices", "Healthcare", "Advertisements", "Power", 
           "Occupations", "Movies", "Welfare", "Breaking up", "Location", 
           "Justice", "Memories", "Crime", "Violence against women", "Equality", 
            "Housing", "Immigration",     
            # new topics
           "Interracial", "Christians", "muslims", "jews", "e-mail", 
           "phone", "democracy", "Democratic", "Republican", "technology", 
           "leadership", "community", "jury", "police", "inequality", 
           "renting", "Violence", "immigrants", "immigrant", "skilled", 
           "Telemarketing", "SPAM", "skill", "job", "health", "mobile", 
            "ads", "physical", "emotional", "bubble", "rent", "economy", 
            "abuse", "women", "city", "country", "suburban", "dollar", 
            "united states", "laws", "phone", "race", "biracial", "interracial", 
            "marriage", "lyrics", "sexuality", "medicine", "television", "european",
            "home", "protect", "spouse", "language", "cellphone", "money",
            "doctor", "insurance", "cigarettes", "alcohol", "income", "salary",
            "class", "censor", "rating", "programs", "government",
            "relationship", "legal", "event", "life", "safe", "victim", "cops",
            "wage", "illegal"
            ]
topics = list(set(t.lower() for t in topics))
topics_stem = [stem(t) for t in topics]

# add similar topic words
new_topics = []
# for t in topics:
#     if t.encode() in sim_dict['w']:
#         new_topics.extend([w.decode() for w in sim_dict['w'][t.encode()]])
topics.extend(new_topics)

In [ ]:
len(topics)

In [ ]:
topics_details = get_details_for_words(topics, en_common_words, en_word_utt_count, 
                                       min_dev_freq=5, 
                                       max_dev_freq=10000, 
                                       min_train_freq=10, 
                                       min_len=1)
print("total words meeting criteria = {0:d}".format(len(topics_details["words"])))

In [ ]:
_, _ = get_duration(topics_details["train_utts"], key="fisher_train"), get_duration(topics_details["dev_utts"], key="fisher_dev")

In [ ]:
print("\n".join(list(topics_details["words"].keys())))

In [ ]:
topics_details['words']

In [ ]:
bow_topics_vocab = create_vocab(topics_details)

In [ ]:
haha = pickle.load(open(os.path.join(m_cfg['data_path'], "bow_topics_vocab.dict"), "rb"))

In [ ]:
len(set(haha['w2i']) & set(bow_topics_vocab['w2i'].keys()))

In [ ]:
pickle.dump(bow_topics_vocab, open(os.path.join(m_cfg['data_path'], "bow_topics_vocab.dict"), "wb"))

### Task 3 - crises terms as keywords

In [ ]:
crises_lex_fname = "../criseslex/CrisisLexLexicon/CrisisLexRec.txt"

In [ ]:
crises = set()
with open(crises_lex_fname, "r") as in_f:
    for line in in_f:
        crises.update(line.strip().split())
crises = list(crises)
crises_stem = [stem(w) for w in crises]

# new_crises = []
# for t in crises:
#     if t.encode() in sim_dict['w']:
#         new_crises.extend([w.decode() for w in sim_dict['w'][t.encode()]])
# crises.extend(new_crises)

In [ ]:
len(crises)

In [ ]:
crises_details = get_details_for_words(crises, en_common_words, en_word_utt_count,
                                       min_dev_freq=10, 
                                       max_dev_freq=1000, 
                                       min_train_freq=100, 
                                       min_len=1)
print("total words meeting criteria = {0:d}".format(len(crises_details["words"])))

In [ ]:
_, _ = get_duration(crises_details["train_utts"], key="fisher_train"), get_duration(crises_details["dev_utts"], key="fisher_dev")

In [ ]:
en_word_utt_count.keys()

In [ ]:
len(en_word_utt_count['dev_utts']['people'])

In [ ]:
print("\n".join(list(crises_details["words"].keys())))

In [ ]:
crises_details["words"]

In [ ]:
bow_crises_vocab = create_vocab(crises_details)

In [ ]:
pickle.dump(bow_crises_vocab, open(os.path.join(m_cfg['data_path'], "bow_crises_vocab.dict"), "wb"))

### More Crisis

In [ ]:
crises_details = get_details_for_words(crises, en_common_words, en_word_utt_count,
                                       min_dev_freq=5, 
                                       max_dev_freq=1000, 
                                       min_train_freq=50, 
                                       min_len=1)
print("total words meeting criteria = {0:d}".format(len(crises_details["words"])))

In [ ]:
_, _ = get_duration(crises_details["train_utts"], key="fisher_train"), get_duration(crises_details["dev_utts"], key="fisher_dev")

In [ ]:
en_word_utt_count.keys()

In [ ]:
len(en_word_utt_count['dev_utts']['people'])

In [ ]:
print("\n".join(list(crises_details["words"].keys())))

In [ ]:
crises_details["words"]

In [ ]:
bow_crises_vocab = create_vocab(crises_details)

In [ ]:
pickle.dump(bow_crises_vocab, open(os.path.join(m_cfg['data_path'], "bow_crises_vocab_more.dict"), "wb"))

### Task Spanish - 500 randomly selected frequent words, minor filtering

In [ ]:
min_dev_freq=18
max_dev_freq=10000
min_train_freq=200
min_len=5

In [ ]:
terms_of_interest = get_details_for_words(es_common_words,
                                          es_common_words,
                                          es_word_utt_count,
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq, 
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(terms_of_interest["words"])))

In [ ]:
_, _ = get_duration(terms_of_interest["train_utts"], key="fisher_train"), get_duration(terms_of_interest["dev_utts"], key="fisher_dev")

In [ ]:
random.seed("hmm")
sample_terms = random.sample(list(terms_of_interest["words"].keys()), 
                             min(len(terms_of_interest["words"].keys()), len(terms_of_interest["words"].keys())))

In [ ]:
# sample_terms = ["bueno"]

In [ ]:
sample_terms_details = get_details_for_words(sample_terms,
                                             es_common_words,
                                             es_word_utt_count,
                                              min_dev_freq=min_dev_freq, 
                                              max_dev_freq=max_dev_freq, 
                                              min_train_freq=min_train_freq, 
                                              min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(sample_terms_details["words"])))

In [ ]:
_, _ = get_duration(sample_terms_details["train_utts"], key="fisher_train"), get_duration(sample_terms_details["dev_utts"], key="fisher_dev")

In [ ]:
sample_terms

In [ ]:
# sample_terms_details

In [ ]:
bow_es_top_words_vocab = create_vocab(sample_terms_details)

In [ ]:
bow_es_top_words_vocab.keys()

In [ ]:
sample_terms_details.keys()

In [ ]:
sample_terms_details["train_utts"][:10]

In [ ]:
sample_terms_details['words']

In [ ]:
sample_terms_details['words']['bueno']

In [ ]:
bow_es_top_words_vocab['freq'][b'bueno'], bow_es_top_words_vocab['freq_dev'][b'bueno']

In [ ]:
pickle.dump(bow_es_top_words_vocab, open(os.path.join(m_cfg['data_path'], "bow_es_100word_vocab.dict"), "wb"))

In [ ]:
pickle.dump(bow_es_top_words_vocab, open(os.path.join(m_cfg['data_path'], "bow_es_1word_vocab.dict"), "wb"))

In [ ]:
!su s1444673

In [ ]:
pickle.dump(bow_es_top_words_vocab, open(os.path.join(m_cfg['data_path'], "bow_es_100word_vocab.dict"), "wb"))

In [ ]:
pickle.dump(bow_es_top_words_vocab, open(os.path.join(m_cfg['data_path'], "bow_es_top_words_vocab.dict"), "wb"))

In [ ]:
sample_terms_details['words']['colorado']

In [ ]:
sorted([(w, sample_terms_details['words'][w]['train']) 
       for w in sample_terms_details['words']], reverse=True, key=lambda t: t[1])[:20]

In [ ]:
sorted([(w, sample_terms_details['words'][w]['dev']) 
       for w in sample_terms_details['words']], reverse=True, key=lambda t: t[1])[:20]

In [ ]:
m_cfg['data_path']

In [ ]:
!ls fbanks_80dim_nltk

In [ ]:
m_cfg["sim_dict"]

In [ ]:
sim_dict = pickle.load(open(os.path.join(m_cfg['data_path'], 'pre_trained_sim.dict'), "rb"))

In [ ]:
len([i for i in sim_dict['w'].values() if len(i)>1])

In [ ]:
sim_dict['w'][b'sure']

In [ ]:
sim_dict['w']

In [ ]:
mix_sim_dict = pickle.load(open(os.path.join(m_cfg['data_path'], 'mix_sim.dict'), "rb"))

In [ ]:
len([i for i in mix_sim_dict['w'].values() if len(i)>1])

In [ ]:
for w, i in [(w.decode(),[j.decode() for j in i]) for w, i in mix_sim_dict['w'].items() if len(i) > 2]:
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
pre_sim_dict = pickle.load(open(os.path.join(m_cfg['data_path'], 'pre_trained_sim.dict'), "rb"))

In [ ]:
len([i for i in pre_sim_dict['w'].values() if len(i)>1])

In [ ]:
for w, i in [(w.decode(),[j.decode() for j in i]) for w, i in pre_sim_dict['w'].items() if len(i) > 2]:
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
pre_sim_dict['w'][b'sure']

In [ ]:
sim_dict = pickle.load(open(os.path.join(m_cfg['data_path'], 'sim.dict'), "rb"))

In [ ]:
len([i for i in sim_dict['w'].values() if len(i)>1])

In [ ]:
for w, i in [(w.decode(),[j.decode() for j in i]) for w, i in sim_dict['w'].items() if len(i) > 2]:
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
pre_words = [w for w, i in pre_sim_dict['w'].items() if len(i)>1]
fisher_words = [w for w, i in sim_dict['w'].items() if len(i)>1]

In [ ]:
len(pre_words), len(fisher_words)

In [ ]:
pre_only = set(pre_words) - set(fisher_words)
fisher_only = set(fisher_words) - set(pre_words)
common_only = set(pre_words) & set(fisher_words)

In [ ]:
len(pre_only), len(fisher_only),  len(common_only), (len(set(pre_words) | set(fisher_words)))

In [ ]:
for w, i in sorted([(w.decode(),[j.decode() for j in i]) for w, i in pre_sim_dict['w'].items() if len(i) > 2 and w in pre_only]):
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
for w, i in sorted([(w.decode(),[j.decode() for j in i]) for w, i in sim_dict['w'].items() if len(i) > 2 and w in fisher_only]):
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
for w, i in sorted([(w.decode(),[j.decode() for j in i]) for w, i in pre_sim_dict['w'].items() 
                    if (len(i) > 1 and w in common_only)]):
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
for w, i in sorted([(w.decode(),[j.decode() for j in i]) for w, i in sim_dict['w'].items() 
                    if (len(i) > 1 and w in common_only)]):
    print(w, "  & ", ", ".join(set(i)-set([w])), " \\\\")

In [ ]:
def check_word(curr_set, word_type, max_len=1):
    found_count = 0
    eng_tokens = []
    for utt in map_dict[curr_set]:
        if word_type.encode() in map_dict[curr_set][utt]["es_w"] and len(map_dict[curr_set][utt]["es_w"]) <= max_len:
            found_count+=1
            if curr_set == "fisher_train":
                eng_tokens.append(" ".join([w.decode() for w in map_dict[curr_set][utt]["en_w"]]))
            else:
                for r in map_dict[curr_set][utt]["en_w"]:
                    eng_tokens.append(" ".join([w.decode() for w in r]))
    print(found_count, len(map_dict[curr_set]), "{0:.2f}".format(found_count / len(map_dict[curr_set]) * 100))
    print(len(set(eng_tokens)))
    return Counter(eng_tokens)

In [ ]:
t = check_word("fisher_train", "si", 1)
t

In [ ]:
d = check_word("fisher_dev", "mhm", 1)
d

In [ ]:
t.most_common(5)

In [ ]:
", ".join(set([i[0] for i in t.most_common(10)]) and set([i[0] for i in d.most_common(10)]))

In [ ]:
# found_count = 0
eng_tokens = []
curr_set= "fisher_train"
for utt in map_dict[curr_set]:
    if b"claro" in map_dict[curr_set][utt]["es_w"] and len(map_dict[curr_set][utt]["es_w"]) <= 1:
        found_count+=1
        eng_tokens.append(" ".join([w.decode() for w in map_dict[curr_set][utt]["en_w"]]))

In [ ]:
found_count, len(map_dict[curr_set]), found_count / len(map_dict[curr_set]) * 100

In [ ]:
Counter(eng_tokens)

In [ ]:
found_count = 0
eng_tokens = []
curr_set= "fisher_dev"
for utt in map_dict[curr_set]:
    if b"claro" in map_dict[curr_set][utt]["es_w"] and len(map_dict[curr_set][utt]["es_w"]) <= 1:
        found_count+=1
        eng_tokens.append(" ".join([w.decode() for w in map_dict[curr_set][utt]["en_w"]]))

In [ ]:
found_count, len(map_dict[curr_set]), found_count / len(map_dict[curr_set]) * 100

In [ ]:
Counter(eng_tokens)

In [ ]:
len(set(eng_tokens))

In [ ]:
sim_dict['w']

### Evaluation dictionaries

In [ ]:
ref_names = ["ref_min-0_max-300.en{0:d}".format(i) for i in range(4)]

In [ ]:
all_ref_names = []
google_ref_names = []
edin_ref_names = []
for r in ref_names:
    google_ref_names.append(os.path.join("google", r))
    edin_ref_names.append(os.path.join("./sp2enw_hyp_search/sp_1.0_l2e-4_rnn-3_drpt-0.5_cnn_96-2-2", r))
#     all_ref_names.append(os.path.join("sp2enw_interspeech/sp_0.33_seed-AB_l2e-4_drpt-0.5", r))
    
all_ref_names = google_ref_names + edin_ref_names    

In [ ]:
def read_all_lines_in_file(fname):
    all_lines = []
    with open(fname, "r") as in_f:
        for line in in_f:
            all_lines.append(set(line.strip().split()))
    return all_lines

In [ ]:
def get_all_words_in_file(fname):
    all_words = []
    with open(fname, "r") as in_f:
        for line in in_f:
            all_words.extend(line.strip().split())
    return set(all_words)

In [ ]:
common_words_ref = get_all_words_in_file(all_ref_names[0])
all_words_ref = get_all_words_in_file(all_ref_names[0])

for r in all_ref_names[1:]:
    common_words_ref &= get_all_words_in_file(r)
    all_words_ref |= get_all_words_in_file(r)

In [ ]:
len(common_words_ref), len(all_words_ref)

In [ ]:
google_common_words_ref = get_all_words_in_file(google_ref_names[0])
google_all_words_ref = get_all_words_in_file(google_ref_names[0])

for r in google_ref_names[1:]:
    google_common_words_ref &= get_all_words_in_file(r)
    google_all_words_ref |= get_all_words_in_file(r)
print(len(google_common_words_ref), len(google_all_words_ref))

In [ ]:
edin_common_words_ref = get_all_words_in_file(edin_ref_names[0])
edin_all_words_ref = get_all_words_in_file(edin_ref_names[0])

for r in edin_ref_names[1:]:
    edin_common_words_ref &= get_all_words_in_file(r)
    edin_all_words_ref |= get_all_words_in_file(r)
print(len(edin_common_words_ref), len(edin_all_words_ref))

In [ ]:
len(google_common_words_ref & edin_common_words_ref)

In [ ]:
common_ref_words = google_common_words_ref & edin_common_words_ref

In [ ]:
len(common_ref_words)

In [ ]:
len(google_all_words_ref - edin_all_words_ref), len(edin_all_words_ref - google_all_words_ref)

In [ ]:
all_ref_lines = {}
for i,r in enumerate(edin_ref_names):
    all_ref_lines[i] = read_all_lines_in_file(r)
    

In [ ]:
words_in_all_refs = []

for i in range(len(all_ref_lines[0])):
    words_ref = all_ref_lines[0][i]
    for j in range(1,4):
        words_ref &= all_ref_lines[j][i]
    words_in_all_refs.append(words_ref)

In [ ]:
word_freq = {}

for ref in words_in_all_refs:
    now_words = ref & common_ref_words - stop_words - es_stop_words
    for w in now_words:
        if w not in word_freq:
            word_freq[w] = 0
        word_freq[w] += 1

In [ ]:
len(word_freq)

In [ ]:
rare_words = [(w,f) for w, f in word_freq.items() if f >= 2 and f <=5 and len(w) >= 8]

### eval 1 - 500 randomly selected frequent words, minor filtering

In [ ]:
min_dev_freq=25
max_dev_freq=10000
min_train_freq=25
max_train_freq=10000
min_len=5

In [ ]:
len(en_common_words)

In [ ]:
en_content_words = ((en_common_words & common_ref_words) - (es_stop_words | stop_words))
en_content_words = {w for w in en_content_words if '¿' not in w and "'" not in w}

In [ ]:
terms_of_interest = get_details_for_words(en_content_words, en_content_words, en_word_utt_count, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq, 
                                          max_train_freq=max_train_freq,
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(terms_of_interest["words"])))

In [ ]:
_, _ = get_duration(terms_of_interest["train_utts"], key="fisher_train"), get_duration(terms_of_interest["dev_utts"], key="fisher_dev")

In [ ]:
random.seed("hmm")
sample_terms = random.sample(list(terms_of_interest["words"].keys()), 
                             min(len(terms_of_interest["words"]), 100))

In [ ]:
sample_terms_details = get_details_for_words(sample_terms, en_common_words, en_word_utt_count,
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq,
                                          max_train_freq=max_train_freq,
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(sample_terms_details["words"])))

In [ ]:
_, _ = (get_duration(sample_terms_details["train_utts"], key="fisher_train"), 
       get_duration(sample_terms_details["dev_utts"], key="fisher_dev"))

In [ ]:
" -- ".join(sample_terms)

In [ ]:
eval_freq_content = create_vocab(sample_terms_details)

In [ ]:
pickle.dump(eval_freq_content, 
            open(os.path.join(m_cfg['data_path'], 
                              "eval_en_freq_vocab.dict"), "wb"))

### eval 2 - 500 randomly selected infrequent words, minor filtering

In [ ]:
min_dev_freq=2
max_dev_freq=10
min_train_freq=2
max_train_freq=25
min_len=5

In [ ]:
len(en_common_words)

In [ ]:
# en_content_words = (en_common_words - (es_stop_words | stop_words))
# en_content_words = {w for w in en_content_words if '¿' not in w}

In [ ]:
len(en_content_words)

In [ ]:
terms_of_interest = get_details_for_words(en_content_words, en_content_words, en_word_utt_count, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq,
                                          max_train_freq=max_train_freq,
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(terms_of_interest["words"])))

In [ ]:
_, _ = get_duration(terms_of_interest["train_utts"], key="fisher_train"), get_duration(terms_of_interest["dev_utts"], key="fisher_dev")

In [ ]:
random.seed("hmm")
sample_terms = random.sample(list(terms_of_interest["words"].keys()), 
                             min(len(terms_of_interest["words"]), 500))

In [ ]:
sample_terms_details = get_details_for_words(sample_terms, en_common_words, en_word_utt_count,
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq,
                                          max_train_freq=max_train_freq,
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(sample_terms_details["words"])))

In [ ]:
_, _ = (get_duration(sample_terms_details["train_utts"], key="fisher_train"), 
       get_duration(sample_terms_details["dev_utts"], key="fisher_dev"))

In [ ]:
" -- ".join(sample_terms)

In [ ]:
eval_content = create_vocab(sample_terms_details)

In [ ]:
pickle.dump(eval_content, 
            open(os.path.join(m_cfg['data_path'], 
                              "eval_en_rare_vocab.dict"), "wb"))

### eval 3 - common es, en words

In [ ]:
min_dev_freq=2
max_dev_freq=10000
min_train_freq=2
max_train_freq=100000
min_len=5

In [ ]:
len(en_common_words)

In [ ]:
# en_content_words = (en_common_words - (es_stop_words | stop_words))
# en_content_words = {w for w in en_content_words if '¿' not in w}

In [ ]:
len(en_content_words)

In [ ]:
es_en_common_words = (es_common_words & en_common_words & common_ref_words)  - (es_stop_words | stop_words)

In [ ]:
len(es_en_common_words)

In [ ]:
terms_of_interest = get_details_for_words(es_en_common_words, en_content_words, en_word_utt_count, 
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq,
                                          max_train_freq=max_train_freq,
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(terms_of_interest["words"])))

In [ ]:
_, _ = get_duration(terms_of_interest["train_utts"], key="fisher_train"), get_duration(terms_of_interest["dev_utts"], key="fisher_dev")

In [ ]:
random.seed("hmm")
sample_terms = random.sample(list(terms_of_interest["words"].keys()), 
                             min(len(terms_of_interest["words"]), 500))

In [ ]:
sample_terms_details = get_details_for_words(sample_terms, en_common_words, en_word_utt_count,
                                          min_dev_freq=min_dev_freq, 
                                          max_dev_freq=max_dev_freq, 
                                          min_train_freq=min_train_freq,
                                          max_train_freq=max_train_freq,
                                          min_len=min_len)
print("total words meeting criteria = {0:d}".format(len(sample_terms_details["words"])))

In [ ]:
_, _ = (get_duration(sample_terms_details["train_utts"], key="fisher_train"), 
       get_duration(sample_terms_details["dev_utts"], key="fisher_dev"))

In [ ]:
" -- ".join(sample_terms)

In [ ]:
eval_content = create_vocab(sample_terms_details)

In [ ]:
pickle.dump(eval_content, 
            open(os.path.join(m_cfg['data_path'], 
                              "eval_en_es_common_vocab.dict"), "wb"))

In [ ]:
es_en_common_words = [w for w in es_common_words & en_common_words if len(w) >= 5]

In [ ]:
len(es_en_common_words)

In [ ]:
es_en_common_words

### Eval crisis


In [ ]:
crises_lex_fname = "../criseslex/CrisisLexLexicon/CrisisLexRec.txt"

In [ ]:
crises = set()
with open(crises_lex_fname, "r") as in_f:
    for line in in_f:
        crises.update(line.strip().split())
crises = list(crises)
crises_stem = [stem(w) for w in crises]

# new_crises = []
# for t in crises:
#     if t.encode() in sim_dict['w']:
#         new_crises.extend([w.decode() for w in sim_dict['w'][t.encode()]])
# crises.extend(new_crises)

In [ ]:
len(crises)

In [ ]:
crises_details = get_details_for_words(crises, (en_common_words & common_ref_words), en_word_utt_count,
                                       min_dev_freq=2, 
                                       max_dev_freq=5000, 
                                       min_train_freq=2,
                                       max_train_freq=5000,
                                       min_len=1)
print("total words meeting criteria = {0:d}".format(len(crises_details["words"])))

In [ ]:
_, _ = get_duration(crises_details["train_utts"], key="fisher_train"), get_duration(crises_details["dev_utts"], key="fisher_dev")

In [ ]:
en_word_utt_count.keys()

In [ ]:
len(en_word_utt_count['dev_utts']['people'])

In [ ]:
len(set(crises_details["words"]))

In [ ]:
print(" -- ".join(list(crises_details["words"].keys())))

In [ ]:
# crises_details["words"]

In [ ]:
bow_crises_vocab = create_vocab(crises_details)

In [ ]:
pickle.dump(bow_crises_vocab, open(os.path.join(m_cfg['data_path'], 
                                                "eval_en_crisis_vocab.dict"), "wb"))

### NLTK wordnet

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet

In [ ]:
en_words = set(en_word_utt_count['train'].keys())

In [ ]:
len(en_words)

In [ ]:
list(en_words)[:5]

In [ ]:
"howdy".encode() in vocab_dict["en_w"]["w2i"]

In [ ]:
syns = {}

for w in tqdm(en_words):
    s = []
#     print(w)
    w_syn = wn.synsets(w)
    for item in w_syn:
        for lm in item.lemma_names():
            if lm in en_words:
                s.append(lm)
        # end for all lemmas
    # end for syns
    s.append(w)
    syns[w] = list(set(s))
# end for all words

In [ ]:
syn_sim = {"w":{}, "i":{}}

for w in tqdm(vocab_dict["en_w"]["w2i"].keys()):
    syn_sim["w"][w] = [w.encode() for w in syns.get(w.decode(), [w.decode()])]
    i = vocab_dict["en_w"]["w2i"][w]
    syn_sim["i"][i] = [vocab_dict["en_w"]["w2i"][j] for j in syn_sim["w"][w]]
    

In [ ]:
list(syn_sim["w"].items())[:5]

In [ ]:
pickle.dump(syn_sim, open(os.path.join(m_cfg['data_path'], "en_syns_train.dict"), "wb"))

In [ ]:
haha = {w:v for w, v in syns.items() if len(v) >= 2}

In [ ]:
len(haha)

In [ ]:
vocab_dict["en_w"]["i2w"][14261]

In [ ]:
sim_keys = set([w for w in sim_dict['w'] if len(sim_dict["w"][w]) >= 2])
syn_keys = set([w for w in syn_sim['w'] if len(syn_sim["w"][w]) >= 2])
print(len(sim_keys), len(syn_keys))

In [ ]:
" -- ".join([w.decode() for w in syn_sim["w"][b'run']])

In [ ]:
syn_keys - sim_keys

In [ ]:
len(syns)

In [ ]:
w = "hello"
w_syn = wn.synsets(w)
for item in w_syn:
    for lm in item.lemma_names():
        if lm in en_words:
            syns[w].append(lm)

In [ ]:
syns = wordnet.synsets('car')

In [ ]:
syn_set = []
for synset in wn.synsets("hello"):
    for item in synset.lemma_names():
        syn_set.append(item)
print(syn_set)

In [ ]:
syns[1].lemmas()[3].name()

In [ ]:
for s in syns:
    print(s.lemmas()[0].name)

In [ ]:
[s.lemmas[0] for s in syns]

### ES, EN common words

In [14]:
def clean_out_str(out_str):
    out_str = out_str.replace("`", "")
    out_str = out_str.replace('"', '')
    out_str = out_str.replace('¿', '')
    out_str = out_str.replace("''", "")
    out_str = out_str.strip()
    return out_str
    

In [11]:
def get_out_str(h):
    out_str = ""
    for w in [i.decode() for i in h]:
        out_str += "{0:s}".format(w) if (w.startswith("'") or w=="n't") else " {0:s}".format(w)

    out_str = clean_out_str(out_str)
    return out_str

In [16]:
def match_es_en(es_words, en_words):
    utt_es_en_stats = {"common":{}, "es": {}, "en": {}}
    
    utt_es_en_stats["es"] = Counter(es_words)
    utt_es_en_stats["en"] = Counter(en_words)
    
    common_es_en = set(utt_es_en_stats["es"].keys()) & set(utt_es_en_stats["en"].keys())
    
    for w in common_es_en:
        utt_es_en_stats["common"][w] = min(utt_es_en_stats["es"][w], utt_es_en_stats["en"][w])
    
    return utt_es_en_stats

In [17]:
all_stop_words = (set(nltk.corpus.stopwords.words("english")) | 
                    set(nltk.corpus.stopwords.words("spanish")))

In [18]:
def show_details(common_results, show=20, min_len=5, max_freq=2, filter_stop=False):
    for k in common_results:
        print("{0:10s} = {1:>10d} types".format(k,len(common_results[k])))
        
    tot = {}
    print("-"*60)
    for k in common_results:
        tot[k] = sum(common_results[k].values())
        print("{0:10s} = {1:>10d} tokens".format(k,tot[k]))
        
    print("-"*60)
    print("common / es = {0:.2f}%".format(tot["common"]/tot["es"]*100))
    print("common / en = {0:.2f}%".format(tot["common"]/tot["en"]*100))
    print("-"*60)
    
    common_words = sorted(common_results["common"].items(), reverse=True, key=lambda t: t[1])
    
    if filter_stop:
        common_words = [(w,c) for w,c in common_words 
                        if w not in all_stop_words and 
                        len(w) >= min_len and "'" not in w and common_results["en"][w] <= max_freq]
        
    non_cv_words = {}
    set_common_words = set(common_results["common"].keys())
    for w,c in common_results["en"].items():
        if ((w not in set_common_words) and (w not in all_stop_words) and 
            (len(w) >= min_len) and ("'" not in w) and 
            (common_results["en"][w] <= max_freq)):
            non_cv_words[w] = common_results["en"][w]
    
    print("total content word types: {0:d}".format(len(common_words)))
    
    print("-"*60)
    tot_content = sum([c for w,c in common_words])
    print("total content tokens: {0:d}".format(tot_content))
    print("common content / es = {0:.2f}%".format(tot_content/tot["es"]*100))
    print("common content / en = {0:.2f}%".format(tot_content/tot["en"]*100))
    print("-"*60)
    print("Top common words")
    for w, c in common_words[:show]:
        print("{0:20s} || {1:10d}".format(w,c))
        
    return {w:c for w,c in common_words}, non_cv_words
        
    

In [19]:
def corpus_level_es_en(set_key, ref_num=0):
    utts_with_common = []
    es_en_stats = {"common":{}, "es": {}, "en": {}}

    for u in tqdm(map_dict[set_key], ncols=80):
        es_words = get_out_str(map_dict[set_key][u]["es_w"]).strip().split()
        if set_key == "fisher_dev":
            en_words = get_out_str(map_dict[set_key][u]["en_w"][ref_num]).strip().split()
        else:
            en_words = get_out_str(map_dict[set_key][u]["en_w"]).strip().split()
        
        utt_es_en_stats = match_es_en(es_words, en_words)
    
        common_words = [w for w,c in utt_es_en_stats["common"].items() 
                            if w not in all_stop_words 
                            and len(w) >= 5 and "'" not in w]
        
        #print(common_words)
        
        if len(common_words) > 0:
            utts_with_common.append(u)
        
        #print(utt_es_en_stats)
        for k in utt_es_en_stats:
            for w in utt_es_en_stats[k]:
                if w not in es_en_stats[k]:
                    es_en_stats[k][w] = 0
                es_en_stats[k][w] += utt_es_en_stats[k][w]
        # end for update counts
    # end for all utts
    
    print("utts with common words: {0:d} / {1:d}".format(len(utts_with_common), 
                                                         len(map_dict[set_key])))
    return es_en_stats

In [ ]:
dev_common = corpus_level_es_en("fisher_dev", 0)

In [ ]:
dev_comm_cont, dev_non_cv = show_details(dev_common, show=10, 
                                         min_len=5, 
                                         max_freq=1, 
                                         filter_stop=True)

In [ ]:
# dev_comm_cont["annie"], dev_common["en"]["annie"]

In [ ]:
sum(dev_comm_cont.values())

In [20]:
train_common = corpus_level_es_en("fisher_train", 0)

100%|████████████████████████████████| 138819/138819 [00:06<00:00, 20604.78it/s]

utts with common words: 18491 / 138819


In [ ]:
train_comm_cont, train_non_cv = show_details(train_common, show=10, 
                                             min_len=5, 
                                             max_freq=10, 
                                             filter_stop=True)

In [ ]:
len(train_comm_cont), len(train_non_cv), len(dev_comm_cont), len(dev_non_cv)

In [ ]:
sum(train_comm_cont.values()), sum(train_non_cv.values()), sum(dev_comm_cont.values()), sum(dev_non_cv.values())

In [ ]:
sum(dev_non_cv.values())

In [ ]:
cv_words = set(train_comm_cont.keys()) & all_dev_en

In [ ]:
non_cv_words = (set(train_non_cv.keys())-in_vocab_rare_words) & all_dev_en

In [ ]:
len(set(train_comm_cont.keys())), len(cv_words), len(non_cv_words)

In [ ]:
cv_noncv_words = {"cv": cv_words, "noncv": non_cv_words}

In [ ]:
pickle.dump(cv_noncv_words, open(os.path.join(m_cfg["data_path"],"cv_noncv.p"), "wb"))

In [ ]:
content_common_words = {"train": train_comm_cont, "dev": dev_comm_cont, 
                        "train_non_cv": train_non_cv, "dev_non_cv": dev_non_cv}

In [ ]:
m_cfg["data_path"]

In [ ]:
pickle.dump(content_common_words, open(os.path.join(m_cfg["data_path"],"common_content.p"), "wb"))

In [ ]:
es_words = get_out_str(map_dict["fisher_dev"]["20051009_182032_217_fsp-B-2"]["es_w"])

In [ ]:
es_words

In [ ]:
# "chemistry" in dev_common["common"]

In [ ]:
# dev_non_cv

In [ ]:
# dev_comm_cont

In [ ]:
oov = set(dev_common["en"].keys()) - set(train_common["en"].keys())

In [ ]:
len(oov)

In [ ]:
t_words = []
with open("interspeech/sp_160hrs/ref_min-0_max-300.en1", "r") as in_f:
    for line in in_f:
        t_words.extend(line.strip().split())

In [ ]:
len(t_words)

In [ ]:
t_words_count = Counter(t_words)

In [ ]:
k1 = set(dev_comm_cont.keys())
k2 = set(t_words_count.keys())

In [ ]:
len(set(dev_comm_cont.keys()) & oov)

In [ ]:
len(set(dev_non_cv.keys()) & oov)

In [ ]:
len(k1), len(k2), len(k1 & k2)

In [ ]:
for w in k1:
    if dev_comm_cont[w] != t_words_count[w]:
        print(w, dev_comm_cont[w], t_words_count[w])

### OOV

In [24]:
t_words = []
with open("interspeech/sp_160hrs/ref_min-0_max-300.en0", "r") as in_f:
    for line in in_f:
        t_words.extend(line.strip().split())

In [ ]:
len(t_words)

In [26]:
t_words_count = Counter(t_words)

In [27]:
all_dev_en = set(t_words_count.keys())
all_train_en = set(train_common["en"].keys())

In [ ]:
len(all_dev_en), len(all_train_en)

In [ ]:
oov = all_dev_en - all_train_en

In [ ]:
len(oov)

In [ ]:
sum([v for w,v in t_words_count.items() if w in oov])

In [ ]:
oov_counter = {w:v for w,v in t_words_count.items() if w in oov}

In [ ]:
pickle.dump(oov_counter, open(os.path.join(m_cfg["data_path"],"oov.p"), "wb"))

### words in google preds not in train

In [ ]:
g_words = []
with open("google/beam_min-0_max-300.en", "r") as in_f:
    for line in in_f:
        g_words.extend(line.strip().split())

In [ ]:
len(g_words)

In [ ]:
g_word_counter = Counter(g_words)

In [ ]:
len(g_word_counter)

In [ ]:
g_new_words = set(g_word_counter.keys()) - all_train_en

In [ ]:
len(g_new_words)

In [ ]:
sum([v for w,v in g_word_counter.items() if w in g_new_words])

In [ ]:
"craigslists" in all_train_en

In [ ]:
g_new_words

In [ ]:
sum([v for w,v in g_word_counter.items() if w in (g_new_words & all_dev_en)])

In [ ]:
g_new_words & oov

In [ ]:
g_new_words & oov

In [ ]:
g_new_words & oov

In [ ]:
sum([v for w,v in g_word_counter.items() if w in (g_new_words & oov)])

In [ ]:
word_lists = {}

In [ ]:
len(train_common["en"])

In [34]:
def filter_train_words(word_freq, show=5, min_len=5, min_freq=1, max_freq=10):
    tot = sum(word_freq.values())
    print("-"*60)    
    print("total types={0:>10d}, tokens = {1:>10d}".format(len(word_freq), tot))
    print("-"*60)    
    
    common_words = sorted(word_freq.items(), reverse=True, key=lambda t: t[1])
    
    common_words = [(w,c) for w,c in common_words 
                    if w not in all_stop_words and 
                    len(w) >= min_len and 
                    "'" not in w and 
                    word_freq[w] >= min_freq and
                    word_freq[w] <= max_freq]
    
    print("-"*60)
    print("total filter word types: {0:d}".format(len(common_words)))
    
    tot_content = sum([c for w,c in common_words])
    print("total filter tokens: {0:d}".format(tot_content))
    print("filter content / en = {0:.2f}%".format(tot_content/tot*100))
    print("-"*60)
    
    print("-"*60)    
    in_dev = all_dev_en & set([w for w,c in common_words])
    in_dev_tokens = sum([t_words_count[w] for w in in_dev])
    print("total filter word types in dev: {0:d}, tokens: {1:d}".format(len(in_dev), in_dev_tokens))
    print("-"*60)    
    
    
    sel_words = {w:c for w,c in common_words if w in in_dev}
    print("Top common words")
    for w, c in sorted(sel_words.items(), reverse=True, key=lambda t:t[1])[:show]:
        print("{0:20s} || {1:10d} || {2:10d}".format(w,c, t_words_count[w]))
        
    return sel_words

In [35]:
train_rare = filter_train_words(train_common["en"], min_len=5, min_freq=1, max_freq=10)

NameError: name 'train_common' is not defined

In [ ]:
train_medium = filter_train_words(train_common["en"], min_len=5, min_freq=25, max_freq=100)

In [28]:
train_frequent = filter_train_words(train_common["en"], min_len=5, min_freq=150, max_freq=100000)

------------------------------------------------------------
total types=     18140, tokens =    1440914
------------------------------------------------------------
------------------------------------------------------------
total filter word types: 386
total filter tokens: 199438
filter content / en = 13.84%
------------------------------------------------------------
------------------------------------------------------------
total filter word types in dev: 365, tokens: 5167
------------------------------------------------------------
Top common words
right                ||       7249 ||        163
think                ||       6676 ||        120
people               ||       6522 ||        183
really               ||       4265 ||         72
things               ||       3058 ||         91


In [ ]:
freq_tables = {"rare": train_rare, "medium": train_medium, "frequent": train_frequent}

In [ ]:
pickle.dump(freq_tables, open(os.path.join(m_cfg["data_path"],"word_freq.p"), "wb"))

In [36]:
haha = pickle.load(open(os.path.join(m_cfg["data_path"],"word_freq.p"), "rb"))

In [38]:
sorted(haha["medium"].items(), reverse=True, key=lambda t: t[1])

[('marry', 100),
 ('needed', 100),
 ('building', 100),
 ('idaho', 100),
 ('hardly', 99),
 ('knowing', 99),
 ('train', 99),
 ('light', 99),
 ('bigger', 98),
 ('supposed', 98),
 ('colorado', 98),
 ('georgia', 98),
 ('students', 98),
 ('cheap', 98),
 ('talks', 97),
 ('treat', 97),
 ('weird', 97),
 ('loved', 97),
 ('guilty', 97),
 ('student', 97),
 ('studies', 96),
 ('behind', 96),
 ('ricans', 96),
 ('shows', 96),
 ('spoken', 95),
 ('eighteen', 95),
 ('connecticut', 95),
 ('involved', 95),
 ('economy', 95),
 ('teacher', 95),
 ('jewish', 95),
 ('justice', 94),
 ('community', 93),
 ('looked', 93),
 ('thinks', 93),
 ('reggaeton', 93),
 ('supposedly', 93),
 ('fighting', 92),
 ('nebraska', 92),
 ('killed', 92),
 ('relation', 92),
 ('illegal', 92),
 ('related', 91),
 ('communicate', 91),
 ('putting', 91),
 ('stopped', 91),
 ('twice', 91),
 ('order', 91),
 ('explain', 90),
 ('leaving', 90),
 ('computers', 90),
 ('types', 90),
 ('emergency', 90),
 ('judge', 90),
 ('situations', 89),
 ('longer', 89

In [29]:
haha["frequent"]

NameError: name 'haha' is not defined

In [ ]:
len(haha["frequent"])

In [ ]:
hehe = pickle.load(open("./fbanks_80dim_nltk/bow_top_100_words_vocab.dict", "rb"))

In [39]:
train_1word = {'music': 2191}
train_fewwords = {'music': 2191, 'person': 1637, 'religion': 1005, 'school': 675, 'church': 468}
train_church = {'church': 468}
train_rare_suffer = {'boring': 66}

In [40]:
train_1word

{'music': 2191}

In [ ]:
len(hehe["w2i"])

In [30]:
def create_new_vocab(words_list):
    out = {"w2i":{}, "i2w":{}, "freq":{}}
    START_VOCAB = [PAD, GO, EOS, UNK]
    for w in START_VOCAB:
        out['w2i'][w] = len(out["w2i"])
        out["freq"][w] = 1
    #for w in words_list['words']:
    sorted_w = sorted(words_list.items(), reverse=True, key=lambda t: t[1])
    for w in sorted_w:
        encoded_word = w[0].encode()
        out["w2i"][encoded_word] = len(out["w2i"])
        out["freq"][encoded_word] = w[1]

    out["i2w"] = {val:key for key, val in out["w2i"].items()}
    return out

In [27]:
frequent_bow = create_new_vocab(haha["frequent"])

NameError: name 'haha' is not defined

In [ ]:
pickle.dump(frequent_bow, open(os.path.join(m_cfg['data_path'], "bow_frequent.en"), "wb"))

In [ ]:
medium_bow = create_new_vocab(haha["medium"])

In [ ]:
pickle.dump(medium_bow, open(os.path.join(m_cfg['data_path'], "bow_medium.en"), "wb"))

In [ ]:
rare_bow = create_new_vocab(haha["rare"])

In [ ]:
pickle.dump(rare_bow, open(os.path.join(m_cfg['data_path'], "bow_rare.en"), "wb"))

In [41]:
oneword_bow = create_new_vocab(train_1word)

In [43]:
pickle.dump(oneword_bow, open(os.path.join(m_cfg['data_path'], "bow_oneword.en"), "wb"))

In [28]:
fewwords_bow = create_new_vocab(train_fewwords)

In [29]:
pickle.dump(fewwords_bow, open(os.path.join(m_cfg['data_path'], "bow_fewwords.en"), "wb"))

In [31]:
church_bow = create_new_vocab(train_church)

In [32]:
pickle.dump(church_bow, open(os.path.join(m_cfg['data_path'], "bow_church.en"), "wb"))

In [40]:
suffer_bow = create_new_vocab(train_rare_suffer)

In [41]:
pickle.dump(suffer_bow, open(os.path.join(m_cfg['data_path'], "bow_suffer.en"), "wb"))